In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from numpy import asarray

In [5]:
df_trg = pd.read_csv('training.csv')
df_trg.head()

,row_id,time,x,y,direction,congestion
0,0,1991-04-01 00:00:00,0,0,EB,70
1,1,1991-04-01 00:00:00,0,0,NB,49
2,2,1991-04-01 00:00:00,0,0,SB,24
3,3,1991-04-01 00:00:00,0,1,EB,18
4,4,1991-04-01 00:00:00,0,1,NB,60


In [6]:
type(df_trg.time[0])

str

In [7]:
route_train = df_trg['x'].astype(str) + df_trg['y'].astype(str) + df_trg['direction']
df_trg.insert(2, 'route', route_train )

In [8]:
df_trg = df_trg.drop(['time', 'x', 'y', 'direction', 'row_id'], axis = 1)

In [9]:
df_trg.head()

,route,congestion
0,00EB,70
1,00NB,49
2,00SB,24
3,01EB,18
4,01NB,60


In [10]:
ct = asarray(df_trg['route'])
ct

array(['00EB', '00NB', '00SB', ..., '21NW', '21SB', '21SE'], dtype=object)

In [11]:
oe = OrdinalEncoder()
df_trg['route'] = oe.fit_transform(ct.reshape(-1,1))

In [12]:
def pairing(df_tr, seq_len=195):

    x = []
    y = []

    for i in range(0,(df_tr.shape[0] - seq_len+1), seq_len+1 ): # range is reduced by the len of seq + 1 so that we do not go out of bounds
                                                            # we step for that same amount of steps as the seq_len
        seq = np.zeros( (seq_len, df_tr.shape[1]) ) #creating a matrix of zeros with the shape of seq_len and the number of columns of the data
        
        for j in range(seq_len):  # filling the matrix with the data

            seq[j] = df_tr.values[i+j]  # filling the matrix with the data we add i to make the jump of the seq_len

        x.append(seq.flatten())  # flattening the matrix and appending it to the x list
        y.append( df_tr["congestion"][i+seq_len] )  # appending the target to the y list 

    return np.array(x), np.array(y)

#print(data.shape)

x, y = pairing(df_trg)

In [ ]:
xtr =x
ytr = y

In [ ]:
xtr

In [ ]:
x.shape

In [ ]:
ytr.shape

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(test_size=15)
for train_index, test_index in tscv.split(x):
    x_train, x_val = x[train_index], x[test_index]
    y_train, y_val = y[train_index], y[test_index]

In [ ]:
x_train.shape

In [ ]:
x_val.shape

In [ ]:
y_train.shape

In [ ]:
y_val.shape

In [ ]:
from os import pardir
import pandas as pd
import numpy as np

from sklearn.tree          import DecisionTreeRegressor
from sklearn.ensemble      import RandomForestRegressor
from sklearn.ensemble      import ExtraTreesRegressor
from sklearn.ensemble      import AdaBoostRegressor
from sklearn.ensemble      import GradientBoostingRegressor
from xgboost               import XGBRegressor
from lightgbm              import LGBMRegressor
from catboost              import CatBoostRegressor
from sklearn.linear_model  import LinearRegression
from sklearn.svm           import SVR
from sklearn import metrics 
from sklearn.preprocessing import StandardScaler
import time

In [ ]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_val = sc.transform(x_val)

In [ ]:
regressors = {
   "Decision Tree": DecisionTreeRegressor(),
   "Extra Trees":   ExtraTreesRegressor(n_estimators=100),
   "Random Forest": RandomForestRegressor(n_estimators=100),
   "AdaBoost":      AdaBoostRegressor(n_estimators=100),
   "Skl GBM":       GradientBoostingRegressor(n_estimators=100),
   "XGBoost":       XGBRegressor(n_estimators=100),
   "LightGBM":      LGBMRegressor(n_estimators=100),
   "CatBoost":      CatBoostRegressor(n_estimators=100),
  'Linear Regression' : LinearRegression(),
  'svr'              :  SVR()
}

In [ ]:
from sklearn import model_selection 
from sklearn import metrics


In [ ]:
# from sklearn.model_selection import GridSearchCV
# svr = SVR()
# params = {'C' : [1,50], 'kernel' : ('linear', 'rbf', 'poly', 'sigmoid'), 'degree' : [3, 6], 'gamma' : ('scale','auto')}
# grid_clf = GridSearchCV(svr, params)

# grid_clf.fit(x_train, y_train)


In [ ]:
# skf = model_selection.StratifiedKFold(
#     n_splits= 5, shuffle=True, random_state=0
# )




# results = pd.DataFrame({'Model': [], 'MSE': [], 'MAE': [], " % error": [], 'Time': []})

# rang = abs(y_train.max()) - abs(y_train.min())

# for model_name, model in regressors.items():
#     start_time = time.time()
   
        
#     # TRAIN AND GET PREDICTIONS USING cross_val_predict() and x,y
#     pred = model_selection.cross_val_predict(model, xtr, ytr, cv = skf)

#     total_time = time.time() - start_time

#     results = results.append({"Model":    model_name,
#                               "MSE": metrics.mean_squared_error(y, pred),
#                               "MAE": metrics.mean_absolute_error(y, pred),
#                               " % error": metrics.mean_squared_error(y, pred) / rang,
#                               "Time":     total_time},
#                               ignore_index=True)
                              
                              



# results_ord = results.sort_values(by=['MSE'], ascending=True, ignore_index=True)
# results_ord.index += 1 
# results_ord.style.bar(subset=['MSE', 'MAE'], vmin=0, vmax=100, color='#5fba7d')




In [ ]:
results = pd.DataFrame({'Model': [], 'MSE': [], 'MAE': [], " % error": [], 'Time': []})
rang = abs(y_train.max()) - abs(y_train.min())
for model_name, model in regressors.items():
    
    start_time = time.time()
    model.fit(x_train, y_train)
    total_time = time.time() - start_time
        
    pred = model.predict(x_val)
    
    results = results.append({"Model":    model_name,
                              "MSE": metrics.mean_squared_error(y_val, pred),
                              "MAE": metrics.mean_absolute_error(y_val, pred),
                              " % error": metrics.mean_squared_error(y_val, pred) / rang,
                              "Time":     total_time},
                              ignore_index=True)
### END SOLUTION


results_ord = results.sort_values(by=['MSE'], ascending=True, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['MSE', 'MAE'], vmin=0, vmax=100, color='#5fba7d')